In [2]:
from pathlib import Path
import pandas as pd

import starfile


In [46]:
run_data_file = Path("/mnt/scratch/ribosomes/kas_k44a/artiax_cleaning/bin6/full_artiax_run_data.star")
output_directory = Path("/mnt/scratch/ribosomes/kas_k44a/artiax_cleaning/bin6")


run_data_star = starfile.read(run_data_file)
run_data_optics = run_data_star["optics"]
run_data_particles = run_data_star["particles"]

In [47]:
tomo_stars = {}
for tomo_name in run_data_particles["rlnTomoName"].unique():
    tomo_stars[tomo_name] = {
        "optics" : run_data_optics,
        "particles" : run_data_particles[run_data_particles["rlnTomoName"] == tomo_name],
    }


In [48]:
for tomo_name in tomo_stars.keys():
    output_filename = f"{tomo_name}_run_data.star"
    starfile.write(tomo_stars[tomo_name], output_directory / output_filename, overwrite=True)


In [59]:
artiax_dict = {}
full_artia_dict = {}
for artiax_star in sorted(output_directory.glob("artiax*run_data.star")):
    artiax_name = artiax_star.name.split("_run")[0][7:]
    artiax_dict[artiax_name] = starfile.read(artiax_star)

    # Make sure rlnTomoName is correct for the new particles
    artiax_dict[artiax_name]["particles"]["rlnTomoName"] = artiax_name

    # If optics was not already added to full_artia_dict, add it
    if "optics" not in full_artia_dict.keys():
        full_artia_dict["optics"] = artiax_dict[artiax_name]["optics"]
    
    # Add particles to full_artia_dict
    if "particles" not in full_artia_dict.keys():
        full_artia_dict["particles"] = artiax_dict[artiax_name]["particles"]
    else:
        full_artia_dict["particles"] = pd.concat(
            [full_artia_dict["particles"], artiax_dict[artiax_name]["particles"]],
            ignore_index=True,
        )

In [ ]:
# if a rlnObjectnumber from df1 is not in df2, add it to df2.
for rlnObjectNumber in artiax_dict["particles"]["rlnObjectNumber"].unique():
    if rlnObjectNumber not in run_data_star["particles"]["rlnObjectNumber"].unique():
        run_data_star["particles"] = run_data_star["particles"].append(
            artiax_dict["particles"][artiax_dict["particles"]["rlnObjectNumber"] == rlnObjectNumber]
        )

for rlnObjectNumber in run_data_star["particles"]["rlnObjectNumber"].unique():
    if rlnObjectNumber not in artiax_dict["particles"]["rlnObjectNumber"].unique():
        # Remove the row from run_data_star using .drop and reindex
        run_data_star["particles"] = run_data_star["particles"].drop(
            run_data_star["particles"][run_data_star["particles"]["rlnObjectNumber"] == rlnObjectNumber].index
        ).reset_index(drop=True)

# remove the rlnObjectNumber column from run_data_star
run_data_star["particles"] = run_data_star["particles"].drop(columns=["rlnObjectNumber"])


In [60]:

# If rlnObjectNumber does not have a unique value for each row, reset it
if not full_artia_dict["particles"]["rlnObjectNumber"].is_unique:
    with open(output_directory / "max_rlnObjectNumber.txt", "r") as f:
        max_object_number = int(f.read())
    # for rows other than index 0, if the rlnObjectNumber is 0.0, set it to max_object_number + 1 and increment max_object_number
    new_max_object_number = max_object_number
    for row in full_artia_dict["particles"].index[1:]:
        if full_artia_dict["particles"].loc[row, "rlnObjectNumber"] == 0.0:
            new_max_object_number = new_max_object_number + 1
            full_artia_dict["particles"].loc[row, "rlnObjectNumber"] = new_max_object_number


In [66]:
# print max rlnObjectNumber to stdout
full_artia_dict["particles"]["rlnObjectNumber"].max()

9944.0

In [68]:
starfile.write(full_artia_dict, output_directory / "full_artiax_run_data.star", overwrite=True)
with open(output_directory / "max_rlnObjectNumber.txt", "w") as f:
    f.write(f"{full_artia_dict['particles']['rlnObjectNumber'].max()}")

,rlnOpticsGroup,rlnOpticsGroupName,rlnSphericalAberration,rlnVoltage,rlnTomoTiltSeriesPixelSize,rlnCtfDataAreCtfPremultiplied,rlnImageDimensionality,rlnTomoSubtomogramBinning,rlnImagePixelSize,rlnImageSize
0,1,opticsGroup1,2.7,300.0,1.0825,1,3,6.0,6.495,72
